In [3]:
len("The current position state of the robotic arm's end gripper is as follows: {Joint_0: 35, Joint_1: 401, Joint_2: 20, Joint_3: -1919, Joint_4: 37, Joint_5: 2337, Joint_6: -766, Joint_7: 40")

186

In [4]:
import enum
import json

def dataset_cmp(dsa, dsb):
    with open(dsa, 'r') as f:
        data_a = json.load(f)
    with open(dsb, 'r') as f:
        data_b = json.load(f)
    for idx, data in enumerate(data_a):
        if data['response'] == "{0 0 0 0 0 0 1}":
            if data_b[idx]['query'][:186] != data['query'][:186]:
                print(f"Mismatch at index {idx}: {data['query']} != {data_b[idx]['query']}")
            # if data_b[idx]['response'] != data['response']:
            #     print(f"Mismatch at index {idx}: {data['response']} != {data_b[idx]['response']}")

dataset_cmp("/mnt/nfs3/caozhe/workspace/ManiSkill/stack_cubes_dual_reproduce/dualcam_dataset_1.json", "/mnt/nfs3/caozhe/workspace/ManiSkill/vlav-project/stack_cubes/dualcam_dataset.json")

Mismatch at index 27: The current position state of the robotic arm's end gripper is as follows: {Joint_0: -78, Joint_1: 418, Joint_2: -91, Joint_3: -2261, Joint_4: 76, Joint_5: 2675, Joint_6: -731, Joint_7: 40}. What action should the robot take to get better completion of instruction: stack the red cube on top of the green one? != The current position state of the robotic arm's end gripper is as follows: {Joint_0: -80, Joint_1: 419, Joint_2: -93, Joint_3: -2265, Joint_4: 76, Joint_5: 2679, Joint_6: -731, Joint_7: 40, Joint_8: 40}. What action should the robot take to get better completion of instruction: stack the red cube on top of the green one?
Mismatch at index 103: The current position state of the robotic arm's end gripper is as follows: {Joint_0: 32, Joint_1: 380, Joint_2: -11, Joint_3: -1985, Joint_4: 17, Joint_5: 2310, Joint_6: -751, Joint_7: 40}. What action should the robot take to get better completion of instruction: stack the red cube on top of the green one? != The cur

In [3]:

import re
import numpy as np

def parse_action_vectors(s: str):
    results = []
    segments = s.strip().split('|')
    for seg in segments:
        seg = seg.strip()
        if not seg:
            continue
        try:
            # 用正则匹配形如 "+0 -8 -2 +3 -4 +13 +1" 的 7 个有符号整数
            matches = re.findall(r'[+-]?\d+', seg)
            if len(matches) != 7:
                results.append(None)
            else:
                results.append(np.array([int(m) for m in matches]))
        except:
            results.append(None)
    return results



test_cases_positive = [
    "+63 +75 -112 +10 -16 -1 +1| +74 +88 -135 +12 -17 +1 +1| +84 +102 -157 +15 -18 +5 +1| +93 +114 -175 +16 -18 +7 +1| +101 +126 -186 +18 -18 +8 +1|",
    # 基础标准格式
    "+1 -2 +3 -4 +5 -6 +7|",
    
    # 多组标准动作向量
    "+0 +0 +0 +0 +0 +0 +1| +1 +2 +3 +4 +5 +6 +7| -1 -2 -3 -4 -5 -6 -7|",
    
    # 包含前后空格
    "   +1 -1 +1 -1 +1 -1 +1|   ",
    
    # 含有多个空格分隔
    "+1    -1   +1 -1    +1 -1 +1|",
    
    # 正负数交替且有+号
    "+1 -1 +2 -2 +3 -3 +4|"
]

test_cases_negative = [
    # 少于7个数字
    "+1 -1 +2|",
    
    # 多于7个数字
    "+1 -1 +2 -2 +3 -3 +4 +5|",
    
    # 含非数字字符
    "+1 -a +2 -2 +3 -3 +4|",
    
    # 含小数
    "+1.0 -1 +2 -2 +3 -3 +4|",
    
    # 中文或其他非 ASCII 字符
    "+1 -1 +2 哈 -2 +3 -3 +4|",
    
    # 字符丢失
    "1 -1 +2 -2 +3 -3 +4|",
    
    # 空段或只有分隔符
    "| | | | | | | |",
    
    # 不完整（无结尾 |）
    "+1 -1 +2 -2 +3 -3 +4"
]

def test_parser():
    print("✅ 正例测试")
    for i, s in enumerate(test_cases_positive, 1):
        res = parse_action_vectors(s)
        print(f"Test {i}: {res}")

    print("\n❌ 反例测试")
    for i, s in enumerate(test_cases_negative, 1):
        res = parse_action_vectors(s)
        print(f"Test {i}: {res}")

test_parser()

✅ 正例测试
Test 1: [array([  63,   75, -112,   10,  -16,   -1,    1]), array([  74,   88, -135,   12,  -17,    1,    1]), array([  84,  102, -157,   15,  -18,    5,    1]), array([  93,  114, -175,   16,  -18,    7,    1]), array([ 101,  126, -186,   18,  -18,    8,    1])]
Test 2: [array([ 1, -2,  3, -4,  5, -6,  7])]
Test 3: [array([0, 0, 0, 0, 0, 0, 1]), array([1, 2, 3, 4, 5, 6, 7]), array([-1, -2, -3, -4, -5, -6, -7])]
Test 4: [array([ 1, -1,  1, -1,  1, -1,  1])]
Test 5: [array([ 1, -1,  1, -1,  1, -1,  1])]
Test 6: [array([ 1, -1,  2, -2,  3, -3,  4])]

❌ 反例测试
Test 1: [None]
Test 2: [None]
Test 3: [None]
Test 4: [None]
Test 5: [array([ 1, -1,  2, -2,  3, -3,  4])]
Test 6: [array([ 1, -1,  2, -2,  3, -3,  4])]
Test 7: []
Test 8: [array([ 1, -1,  2, -2,  3, -3,  4])]


In [4]:
import re
from networkx import non_edges
import numpy as np
def extract_action_vectors(s, vector_length, expected_count=None):
    """
    从字符串 s 中提取动作向量。
    
    参数:
        s (str): 包含若干用 {} 括起来的动作向量字符串。
        vector_length (int): 每个动作向量应包含的整数个数。
        expected_count (int, optional): 期望动作数量，如果提供且不匹配则返回 None。
        
    返回:
        如果 expected_count 不为 None 且与实际提取的动作数不符，则返回 None；
        否则返回长度为提取动作数的列表，列表中每个元素要么是长度为 vector_length 的整数列表，要么是 None（表示该动作格式不合规）。
    """
    contents = re.findall(r'\{([^}]*)\}', s)
    pattern = re.compile(r'^-?\d+(?:\s+-?\d+){' + str(vector_length - 1) + r'}$')
    results = []
    for content in contents:
        raw = content.strip()
        if pattern.match(raw):
            nums = list(map(int, raw.split()))
            results.append(np.array(nums))
        else:
            print(f"Invalid action format: '{raw}'")
            break
    if len(results) > expected_count:
        results = results[:expected_count]
    if len(results) == 0:
        return None
    return results

# 测试样例
test_cases = [
    # 完整且紧凑的两个动作
    ("{1 2 3}{4 5 6}", 3, 2),
    # 两个动作中间有空格
    ("{1 2 3} {4 5 6}", 3, 2),
    # 第一个动作缺少一个数字
    ("{1 2}{4 5 6}", 3, 2),
    # 第二个动作含有非法字符
    ("{1 2 3}{4 a 6}", 3, 2),
    # 动作数量与预期不符
    ("{1 2 3}{4 5 6}{7 8 9}", 3, 2),
]

for s, vec_len, exp in test_cases:
    result = extract_action_vectors(s, vec_len, exp)
    print(f"输入: {s}\n输出: {result}\n")


输入: {1 2 3}{4 5 6}
输出: [array([1, 2, 3]), array([4, 5, 6])]

输入: {1 2 3} {4 5 6}
输出: [array([1, 2, 3]), array([4, 5, 6])]

Invalid action format: '1 2'
输入: {1 2}{4 5 6}
输出: None

Invalid action format: '4 a 6'
输入: {1 2 3}{4 a 6}
输出: [array([1, 2, 3])]

输入: {1 2 3}{4 5 6}{7 8 9}
输出: [array([1, 2, 3]), array([4, 5, 6])]

